<a href="https://colab.research.google.com/github/1uch0/LLM_Udemy_course/blob/main/LLM_RAG_using_Langchain_retrieval_information_from_files.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Expert Knowledge Worker

### A question answering agent that is an expert knowledge worker
### To be used by employees of Insurellm, an Insurance Tech company
### The agent needs to be accurate and the solution should be low cost.

This project will use RAG (Retrieval Augmented Generation) to ensure our question/answering assistant has high accuracy.

In [15]:
#!pip install gradio
#!pip install anthropic
#!pip install langchain
#!pip install -U langchain-community

# imports

import os
import glob
#from dotenv import load_dotenv
import gradio as gradio
from google.colab import userdata
from openai import OpenAI

In [16]:
# imports for langchain

from langchain.document_loaders import DirectoryLoader, TextLoader #Utility classes to load in files
from langchain.text_splitter import CharacterTextSplitter

In [17]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [21]:
# price is a factor for our company, so we're going to use a low cost model
MODEL = "gpt-4o-mini"
db_name = "vector_db"

In [22]:
api_key = userdata.get('Open_AI_API') ##Open_AI_API It is the API key from OPEN_AI saved in your collab
openai = OpenAI(api_key=api_key)
openai_api_key = api_key

OPENAI_MODEL = "gpt-4o-mini"


In [25]:
# Read in documents using LangChain's loaders
# Take everything in all the sub-folders of our knowledgebase



folders = glob.glob("/content/drive/MyDrive/Colab Notebooks/LLM Engineering course/knowledge-base*") #We first get a list of the differtn folders in the kowledge based

# With thanks to CG and Jon R, students on the course, for this fix needed for some users
text_loader_kwargs = {'encoding': 'utf-8'}
# If that doesn't work, some Windows users might need to uncomment the next line instead
# text_loader_kwargs={'autodetect_encoding': True}

#For each folder we get the type of document. And then load it into a directory path
documents = []
for folder in folders:
    doc_type = os.path.basename(folder)
    loader = DirectoryLoader(folder, glob="**/*.md", loader_cls=TextLoader, loader_kwargs=text_loader_kwargs)
    #We bring in all the documents
    folder_docs = loader.load()
    for doc in folder_docs:
        doc.metadata["doc_type"] = doc_type
        #Add this to a list called documents
        documents.append(doc)

In [26]:
len(documents) #31 elemwnts of type document

31

In [27]:
documents[24] #Metada

Document(metadata={'source': '/content/drive/MyDrive/Colab Notebooks/LLM Engineering course/knowledge-base/employees/Jordan Blake.md', 'doc_type': 'knowledge-base'}, page_content="# HR Record\n\n# Jordan Blake\n\n## Summary\n- **Date of Birth:** March 15, 1993  \n- **Job Title:** Sales Development Representative (SDR)  \n- **Location:** Austin, Texas  \n\n## Insurellm Career Progression\n- **2021-06:** Joined Insurellm as an Entry-Level SDR  \n- **2022-02:** Promoted to Junior SDR after exceeding quarterly targets by 25%  \n- **2022-12:** Recognized as SDR of the Month for three consecutive months  \n- **2023-05:** Participated in the Insurellm Leadership Training Program  \n\n## Annual Performance History\n- **2021:** First year at Insurellm; achieved 90% of monthly targets.  \n  - **Feedback:** Strong potential shown in lead generation; needs improvement in follow-up techniques.  \n- **2022:** Achieved 120% of targets; pioneered outreach strategies that increased customer engagement.

In [28]:
#Take the documenrs and divide each document into chyks of characters.
#We specify chunk size: How many characters to fit in each chunk (in sensible boundaries)
#Cunk overlap: we dont want these chunk of characters compeetly sepaarted.

text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
chunks = text_splitter.split_documents(documents)

In [29]:
len(chunks)

123

In [30]:
chunks[4]

Document(metadata={'source': '/content/drive/MyDrive/Colab Notebooks/LLM Engineering course/knowledge-base/contracts/Contract with Greenstone Insurance for Homellm.md', 'doc_type': 'knowledge-base'}, page_content='4. **Payment Terms**: \n   - The Customer shall pay an amount of $10,000 per month for the Standard Tier of the Homellm service.\n   - Payments are due within 30 days of invoicing.\n\n5. **Customization**: Any additional customization requests outside the standard offerings will require a separate agreement and associated costs.\n\n---\n\n## Renewal\n\n1. **Automatic Renewal**: This Agreement will automatically renew for additional one-year terms unless either party provides written notice of termination at least 60 days prior to the end of the current term.\n\n2. **Renewal Terms Review**: Prior to each renewal, the Provider and Customer will review the terms and pricing. Adjustments may be made based on the current features and market conditions.\n\n---\n\n## Features')

In [31]:
doc_types = set(chunk.metadata['doc_type'] for chunk in chunks)
print(f"Document types found: {', '.join(doc_types)}")

Document types found: knowledge-base


In [32]:
for chunk in chunks:
    #Which chunk has CEO on it
    #Smarter about the way it found chunks, not just cjhunks but understanding about the meainignof it
    if 'CEO' in chunk.page_content:
        print(chunk)
        print("_________")

page_content='## Support

1. **Customer Support**: Velocity Auto Solutions will have access to Insurellm’s customer support team via email or chatbot, available 24/7.  
2. **Technical Maintenance**: Regular maintenance and updates to the Carllm platform will be conducted by Insurellm, with any downtime communicated in advance.  
3. **Training & Resources**: Initial training sessions will be provided for Velocity Auto Solutions’ staff to ensure effective use of the Carllm suite. Regular resources and documentation will be made available online.

---

**Accepted and Agreed:**  
**For Velocity Auto Solutions**  
Signature: _____________________  
Name: John Doe  
Title: CEO  
Date: _____________________  

**For Insurellm**  
Signature: _____________________  
Name: Jane Smith  
Title: VP of Sales  
Date: _____________________' metadata={'source': '/content/drive/MyDrive/Colab Notebooks/LLM Engineering course/knowledge-base/contracts/Contract with Velocity Auto Solutions for Carllm.md', '